In [1]:
import subprocess
import os
import base64
from datetime import datetime

In [4]:
def snap(command, exec_env):
    print(command,exec_env)
    snap = subprocess.Popen(["bash","-x",command],
                            stdin =subprocess.PIPE,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True,
                            bufsize=0,
                            cwd='/tmp',
                           env=exec_env)

    # Send ssh commands to stdin
    #snap.stdin.write("uname -a\n")
    #snap.stdin.write("uptime\n")
    #snap.stdin.close()

    # Fetch output
    for line in snap.stdout:
        print(line.strip())

In [ ]:
def get_latest_dataset(context,snap_path):
    metrics_table=os.path.join(snap_path,context.get_param('metrics_table', 'metrics_table'))
    # Get parquet files
    mpath = [os.path.join(metrics_table, file) for file in os.listdir(metrics_table)]
    
    # Get latest filename
    latest = max(mpath, key=os.path.getmtime)
    context.log_result('training_parquet_file',latest)
    context.logger.debug(f'Reading data from: {latest}')
    
    return ""

In [8]:
def handler(context):
    is_simulated = context.get_param('NETAPP_SIM',False)
    NETAPP_MOUNT_PATH = context.get_param('NETAPP_MOUNT_PATH','/netapp')
    print("SIMULATED")
    if is_simulated:
        snap_path = os.path.join(NETAPP_MOUNT_PATH)
        context.log_result('snapVolumeDetails',snap_path)
        context.logger.info('snap volume name: %s'%snap_path)
        get_latest_dataset(context,snap_path) 
        return snap_path
    exec_env={}
    exec_env['MANAGER'] = context.get_param('MANAGER')
    exec_env['svm'] = context.get_param('svm')
    exec_env['email']  = context.get_param('email')
    exec_env['password'] = context.get_param('password')
    exec_env['weid']  = context.get_param('weid')
    exec_env['volume'] = context.get_param('volume')
    exec_env['NETAPP_MOUNT_PATH'] = context.get_param('NETAPP_MOUNT_PATH')
    
    command = os.path.join(context.get_param('APP_DIR'),"snap_cv.sh")
    
    timestamp = datetime.today().strftime("%Y%m%d_%H%M%S")
    exec_env['snapshotName'] = 'kfp_%s' % timestamp
    
    NETAPP_MOUNT_PATH = context.get_param('NETAPP_MOUNT_PATH','/netapp')
    
    snap(command, exec_env)
    
    snap_path = os.path.join(NETAPP_MOUNT_PATH,'.snapshot',exec_env['snapshotName'])
    
    context.log_result('snapVolumeDetails',snap_path)
    context.logger.info('Source Cloud volume name: %s Snapshop: %s' % (exec_env['volume'],exec_env['snapshotName']))
    
    get_latest_dataset(context,snap_path)    